In [ ]:
import iris
import numpy as np
from wildfires.analysis import cube_plotting
from wildfires.data import GFEDv4
from wildfires.utils import ensure_datetime, get_land_mask
from wildfires.variogram import plot_variogram

from empirical_fire_modelling.utils import tqdm

In [ ]:
cube = GFEDv4().cube[0]
_ = cube_plotting(
    np.sqrt(iris.analysis.cartography.area_weights(cube)) / 1000,
    colorbar_kwargs={"label": "approx. cell size (km)"},
)

In [ ]:
def gfed4_variogram(i):
    gfed4 = GFEDv4()
    if i == -1:
        title = "Mean GFED4 BA"
        ba = gfed4.get_mean_dataset().cube
    else:
        ba = gfed4.cube[i]
        title = f"GFED4 BA {ensure_datetime(ba.coord('time').cell(0).point):%Y-%m}"

    ba.data.mask = ~get_land_mask()

    latitudes = ba.coord("latitude").points
    longitudes = ba.coord("longitude").points

    coords = []
    for lon in longitudes:
        for lat in latitudes:
            coords.append((lat, lon))
    coords = np.array(coords)
    ba_flat = ba.data.ravel()

    # Choose random indices.
    valid_indices = np.where(~ba.data.mask.ravel())[0]
    # inds = np.random.default_rng(0).choice(valid_indices, size=(4000,))
    inds = valid_indices

    # print(f"Max N:    {valid_indices.shape[0]:>10d}")
    # print(f"Chosen N: {inds.shape[0]:>10d}")

    fig, ax1, ax2 = plot_variogram(
        coords[inds],
        ba_flat.data[inds],
        bins=50,
        max_lag=2000,
        n_jobs=32,
        n_per_job=6000,
        verbose=True,
    )

    fig.suptitle(f"{title}, {inds.shape[0]} samples (out of {valid_indices.shape[0]})")
    ax1.set_ylabel("semivariance")
    ax2.set_ylabel("N")
    ax2.set_yscale("log")
    ax1.set_xlabel("Lag (km)")

In [ ]:
for i in tqdm(range(-1, 2)):
    gfed4_variogram(i)

In [ ]:
for i in tqdm(range(-1, 36)):
    gfed4_variogram(i)